In [1]:
#   1) Кластеризация фильмов на Кинопоиске для "холодных" рекомендаций похожих фильмов
#   2) Топ жанров по годам (таблица год.жанр.кол-во фильмов) 

In [2]:
# URL: https://www.kinopoisk.ru/top/lists/1/filtr/all/sort/order/perpage/200/page=2/

In [3]:
# 1) библиотеки request, BeautifulSoup, lxml
#    sklearn: kmeans (кластеризация), DesicionTrees (кластеризация), cmeans (кластеризация), EM (кластеризация)
# 2) pandas groupby

In [73]:
from bs4 import BeautifulSoup
from lxml import html
import re

In [127]:
# используем библиотеку requests для вывода html в отдельный файл
import requests
response=requests.get('https://www.kinopoisk.ru/top/lists/1/filtr/all/sort/order/perpage/200/page=2/')
with open('kinopoisk.html', 'wb') as output_file:
  output_file.write(response.text.encode('cp1251'))

In [128]:
# используем библиотеку BeautifulSoup для парсинга

soup=BeautifulSoup(open('kinopoisk.html'),'html.parser')
tags=soup.find('table',{'id':'itemList'})
items=tags.find_all('td',{'class':'news'})
results=[]


In [129]:
for item in items:
    
    # получаем название фильма на русском
    rname=item.find('button').get('data-title')
    rus_name=rname.split()
    name_str=' '.join(rus_name)
    
    
    # необходимо получить хронометраж и год фильма
    data=item.find('div',{'style':'margin-bottom: 9px'})
    film_data=data.find('span').text
            
    # получаем хронометраж фильма
    time=item.find('nobr').text
    time_list=time.split()
    time_str=' '.join(time_list)
    time_num=re.findall('([0-9]*)', time_str)[0]
    time_int=int(time_num)
        
    # получаем год фильма
    year=re.findall('\(\S*[^\d]+', film_data)[0]
    year_clean=year.split()
    year_str=''.join(year_clean)
    year_num=re.findall('([0-9]*)', year_str)[1]
    year_int=int(year_num)
    
    # оставшиеся данные
    missing_data=item.find('span',{'class':'gray_text'}).text.split()
    
    # получаем страну, выпустившую фильм
    country=missing_data[0] 
    country_clean=re.findall('(^[А-Я]+[а-я]*)', country) # данные очищены - имеем только название страны
    country_str=''.join(country_clean)
    
    # получаем жанр фильма
    genre_data=missing_data[1:] 
    genre_str=''.join(genre_data)
    
    # найдем данные, содержащиеся в скобках
    clean_genre=re.split('((^[^()]+))', genre_str)[3:]
    genre_string=''.join(clean_genre)
   
    #оставляем один жанр (первый по счету)
    genre_clean=re.findall('([а-я]*)',genre_string)[1:2]
    genre_str=''.join(genre_clean)

   
    df=(name_str, time_int, year_int, country_str, genre_str)
    results.append(df)

In [130]:
import pandas as pd
df = pd.DataFrame(results)
df.columns=['Name','Time, min','Year','Country','Genre']

In [131]:
df

,Name,"Time, min",Year,Country,Genre
0,Три идиота,170,2009,Индия,драма
1,Трасса 60,116,2001,Канада,фантастика
2,Медведь,94,1988,Франция,драма
3,Песнь моря,93,2014,Ирландия,мультфильм
4,Брат 2,127,2000,Россия,боевик
5,Я – Сэм,132,2001,США,драма
6,Золотая лихорадка,95,1925,США,драма
7,Загадочная история Бенджамина Баттона,166,2008,США,драма
8,Гарри Поттер и узник Азкабана,142,2004,Великобритания,фэнтези
9,Подмена,141,2008,США,триллер


In [133]:
# Сохраняем данный датафрейм в формате CSV для последующего анализа

df.to_csv('kinopoisk.csv')